In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

# DGP for regression

Here we'll show the DGP for regression, using small to medium data sets. 

In [2]:
import numpy as np
import tensorflow as tf
import time

import matplotlib.pyplot as plt
%matplotlib inline 

from gpflow.likelihoods import Gaussian
from gpflow.kernels import RBF, White
from gpflow.mean_functions import Constant
from gpflow.models.sgpr import SGPR, GPRFITC
from gpflow.models.svgp import SVGP
from gpflow.models.gpr import GPR
from gpflow.training import AdamOptimizer, ScipyOptimizer

from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp

from doubly_stochastic_dgp.dgp import DGP
from datasets import Datasets
datasets = Datasets(data_path='data/')

/home/gonzalo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Let's use the kin8nm data set

In [3]:
data = datasets.all_datasets['kin8nm'].get_data()
X, Y, Xs, Ys, Y_std = [data[_] for _ in ['X', 'Y', 'Xs', 'Ys', 'Y_std']]
print('N: {}, D: {}, Ns: {}'.format(X.shape[0], X.shape[1], Xs.shape[0]))
Z_100 = kmeans2(X, 100, minit='points')[0]
Z_500 = kmeans2(X, 500, minit='points')[0]

N: 7372, D: 8, Ns: 820


## Single layer models

Our baseline model is a sparse GP, but since the dataset is small we can also train without minibatches so we'll also compare to a collapsed sparse GP (with analytically optimal $q(\mathbf u)$) which is known as SGPR in GPflow terminology, and we'll also cpmpare to FITC

## DGP models

We'll include a DGP with a single layer here for comparision. We've used a largish minibatch size of $\text{min}(1000, N)$, but it works fine for smaller batches too

In the paper we used 1 sample. Here we'll go up to 10 in celebration of the new implementation (which is much more efficient)

In [4]:
def make_dgp(X, Y, Z, L):
    D = X.shape[1]
    
    # the layer shapes are defined by the kernel dims, so here all hidden layers are D dimensional 
    kernels = []
    for l in range(L):
        kernels.append(RBF(D))
        
    # between layer noise (doesn't actually make much difference but we include it anyway)
    for kernel in kernels[:-1]:
        kernel += White(D, variance=1e-5) 
        
    mb = 1000 if X.shape[0] > 1000 else None 
    model = DGP(X, Y, Z, kernels, Gaussian(), num_samples=10, minibatch_size=mb)

    # start the inner layers almost deterministically 
    for layer in model.layers[:-1]:
        layer.q_sqrt = layer.q_sqrt.value * 1e-5
    
    return model

m_dgp1 = make_dgp(X, Y, Z_100, 1)
m_dgp2 = make_dgp(X, Y, Z_100, 2)
m_dgp3 = make_dgp(X, Y, Z_100, 3)
m_dgp4 = make_dgp(X, Y, Z_100, 4)
m_dgp5 = make_dgp(X, Y, Z_100, 5)

## Prediction

We'll calculate test rmse and likelihood in batches (so the larger datasets don't cause memory problems)

For the DGP models we need to take an average over the samples for the rmse. The `predict_density` function already does this internally


In [5]:
def batch_assess(model, assess_model, X, Y):
    n_batches = max(int(X.shape[0]/1000.), 1)
    lik, sq_diff = [], []
    for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
        l, sq = assess_model(model, X_batch, Y_batch)
        lik.append(l)
        sq_diff.append(sq)
    lik = np.concatenate(lik, 0)
    sq_diff = np.array(np.concatenate(sq_diff, 0), dtype=float)
    return np.average(lik), np.average(sq_diff)**0.5

def assess_single_layer(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch)
    lik = np.sum(norm.logpdf(Y_batch*Y_std, loc=m*Y_std, scale=Y_std*v**0.5),  1)
    sq_diff = Y_std**2*((m - Y_batch)**2)
    return lik, sq_diff 

S = 100
def assess_sampled(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch, S)
    S_lik = np.sum(norm.logpdf(Y_batch*Y_std, loc=m*Y_std, scale=Y_std*v**0.5), 2)
    lik = logsumexp(S_lik, 0, b=1/float(S))
    
    mean = np.average(m, 0)
    sq_diff = Y_std**2*((mean - Y_batch)**2)
    return lik, sq_diff

## Training 

We'll optimize single layer models and using LFBGS and the dgp models with Adam. It will be interesting to compare the result of `m_svgp` compared to `m_dgp1`: if there is a difference it will be down to the optimizer. 

Now for the DGP models:

In [6]:
AdamOptimizer(0.01).minimize(m_dgp2, maxiter=500)
lik, rmse = batch_assess(m_dgp2, assess_sampled, Xs, Ys)
print("Name: m_dgp2 LL: {} RMSE: {}".format(lik, rmse))

Name: m_dgp2 LL:0.9849280605583839 RMSE:0.08678000727811114


In [30]:
S = 100
m, v = m_dgp2.predict_y(Xs, S)
print(m.shape)
lpdf = norm.logpdf(Ys*Y_std, loc=m*Y_std, scale=Y_std*v**0.5)
print(lpdf.shape)

np.mean((logsumexp(lpdf, 0) + np.log(1/S)))

(100, 820, 1)
(100, 820, 1)


0.9841364307469942

In [27]:
m, v = m_dgp2.predict_y(Xs, S)
S_lik = norm.logpdf(Ys*Y_std, loc=m*Y_std, scale=Y_std*v**0.5)
print(S_lik.shape)
lik = logsumexp(S_lik, 0, b=1/float(S)) # Over num samples
print(lik.shape)
np.mean(lik)

(100, 820, 1)
(820, 1)


0.9859163535182438

(100, 820)